## This notebook was forked and edited. The fork was from Katherine Crowson's v-diffusion-pytorch repository. The additions are:
## P. Fishwick, 01/17/22
1. Center installation in Google Drive by mounting it.
2. First run: the installation - store in Drive
3. First run: the pretrained model - store in Drive
4. A loop over a range of skip_timestep (st) and seed values (bottom of notebook) -- this allows you to "batch" run the notebook to create multiple images. There is a --batch_size option in cfg_sample.py and this is NOT used.
5. A collage option -- makes rectangular collages e.g. 3x3, 6x6, 4x7, 2x4
6. Storing resulting images in a zip file and then downloading

In [ ]:
# Load Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#
# discover the assigned GPU
#
!nvidia-smi

In [ ]:
# install requirements
!pip install ftfy regex requests tqdm

In [ ]:
# obtain v-diffusion-pytorch
# run this one time
import os
if (not os.path.isdir('/content/drive/MyDrive/v-diffusion-pytorch')):
  !git clone --recursive https://github.com/metaphorz/v-diffusion-pytorch /content/drive/MyDrive/v-diffusion-pytorch

In [ ]:
#
# Download the diffusion model
# The model list (checkpoints) can be found https://github.com/crowsonkb/v-diffusion-pytorch#model-checkpoints
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda
#
# Save pretrained model in Google Drive for first run (v-diffusion-pytorch/ should not exist)
import os
if (not os.path.isdir('/content/drive/MyDrive/v-diffusion-pytorch/checkpoints')):
  !mkdir /content/drive/MyDrive/v-diffusion-pytorch/checkpoints
  !curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > /content/drive/MyDrive/v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth
# From original 
# !curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > /content/drive/MyDrive/v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

In [ ]:
%cd /content/drive/MyDrive/v-diffusion-pytorch/

In [ ]:
# Run inference loop given the init image for each st and seed value
# Edit the st and range values
import numpy as np
#creates 36 values for seed
#st_start = 0.1
#st_stop = 1.0
#st_step = 0.025
#creates 1 st value
st_start = 0.975
st_stop = 1.0
st_step = 0.5
float_range_array = np.arange(st_start, st_stop, st_step)
st_values = list(float_range_array)
print('# of st_values ', len(st_values))
print('# of seeds ', len(range(0,36)))
print('# of generated images ', len(st_values)*len(range(0,36)))
print('Image Generation ...')
for st_num in st_values:
  for seed_num in range(0,36):
    # enter your own init image
    !python cfg_sample.py --init /content/drive/MyDrive/images/image3.jpg --batch-size 1 -st $st_num --size 512 512 --seed $seed_num

In [ ]:
# save all images in a zip
!zip results/images.zip results/*.png

In [ ]:
# download images.zip
from google.colab import files
files.download('results/images.zip')

In [ ]:
# Optional: Google Drive results/ cleaning
# clean your results folder between notebook runs
# once images.zip has downloaded
#!rm /content/drive/MyDrive/v-diffusion-pytorch/results/images.zip
#!rm /content/drive/MyDrive/v-diffusion-pytorch/results/*.png

In [ ]:
%cd /content/drive/MyDrive/v-diffusion-pytorch/results

In [ ]:
# Optional/Experimental: Generate a collage of all images
# and store in v-diffusion-pytorch/
# Code adapted from: https://holypython.com/python-pil-tutorial/creating-photo-collages/

from PIL import Image, ImageDraw
import glob
lst = sorted(glob.glob("*.png"))
print('check list length ',len(lst))
# to make the collage, you need num_x * num_y .png files
# need not be square
# 9 images to make a 3x3 collage
# 36 images to make a 6x6 collage
# 500 pixels for each side of square component images
num_x = 6
num_y = 6
# choose this to modify the size of each square in the collage
side_square = 200 

collage = Image.new("RGBA", (side_square*num_x,side_square*num_y), color=(255,255,255,255))

file_num = 0
for j in range(0,num_y*side_square,side_square):
    for i in range(0,num_x*side_square,side_square):
        image_file = str(lst[file_num])
        photo = Image.open(image_file).convert("RGBA")
        photo = photo.resize((side_square,side_square))          
        collage.paste(photo, (i,j))
        file_num += 1

display(collage)
collage.save("/content/drive/MyDrive/v-diffusion-pytorch/collage.png")
